In [234]:
import pandas as pd
import string
import os
from glob import glob
import random
import json
import regex as re
from IPython.display import JSON,display,display_json
from tqdm import tqdm
import Levenshtein
from termcolor import colored as clrd
pd.set_option('display.max_colwidth', 300)

In [9]:
key_phrases_excel = "./soure_data/приложение 7 Ключевые фразы по СПГЗ.xlsx"
df = pd.read_excel(key_phrases_excel)
df

,СПГЗ,КПГЗ,Ключевые слова
0,Восстановление газона в рамках обустройства об...,02.03.09 УСТРОЙСТВО ПЛОСКОСТНЫХ ПАРКОВОК И СПЕ...,NaN
1,Восстановление дорожного покрытия в рамках обу...,02.03.09 УСТРОЙСТВО ПЛОСКОСТНЫХ ПАРКОВОК И СПЕ...,NaN
2,"Выкорчевка пней в рамках обустройства объекта,...",02.03.09 УСТРОЙСТВО ПЛОСКОСТНЫХ ПАРКОВОК И СПЕ...,корчев* удален* пней*
3,Выполнение пуско-наладочных работ оборудования...,02.03.09 УСТРОЙСТВО ПЛОСКОСТНЫХ ПАРКОВОК И СПЕ...,NaN
4,Вырубка дерева в в рамках благоустройства терр...,02.03.03.03 ОЗЕЛЕНЕНИЕ ТЕРРИТОРИЙ,"выруб*, дерев*"
...,...,...,...
2182,Уход и восстановление неприжившихся деревьев в...,02.04.04.03.01 ВОССТАНОВЛЕНИЕ ЗЕЛЕНЫХ НАСАЖДЕН...,NaN
2183,Уход и восстановление неприжившихся кустарнико...,02.04.04.03.01 ВОССТАНОВЛЕНИЕ ЗЕЛЕНЫХ НАСАЖДЕН...,NaN
2184,Формовочная обрезка дерева в рамках благоустро...,02.03.03.03 ОЗЕЛЕНЕНИЕ ТЕРРИТОРИЙ,"формор*, обрез*, дерев*"
2185,Формовочная обрезка дерева в рамках ремонтно-в...,02.06.05.05 РАБОТЫ РЕМОНТНО-ВОССТАНОВИТЕЛЬНЫЕ ...,"формор*, обрез*, дерев*"


In [323]:
with open("all_items.json") as f:
    smeta = json.load(f)

In [324]:
key_ph = df["Ключевые слова"].dropna().apply(lambda s: re.sub("[^а-яА-Я]"," ",s).lower()).str.split()

In [332]:
list_poss_matches = []
razdel_poss_matches = []
for pos_ind, pos in enumerate(smeta):
    poss_matches = [(None, -1, -1)]
    for ind, phrs in key_ph.iteritems():
        matched = 0
        for phr in phrs:
            if(phr in pos["Name"].lower()):
                matched+=1
        if(matched):
            poss_matches.append((ind,matched/len(phrs), Levenshtein.ratio(pos["Name"].lower(), df.iloc[ind].СПГЗ)))
    
    poss_matches = sorted(poss_matches,key=lambda x: (-x[1],-x[2]))
    poss_matches = poss_matches[:5]
    
    if(pos_ind != 0  and pos["Section"] != smeta[pos_ind - 1]["Section"]):
        razdel_poss_matches.sort(key=lambda x: x[1][0][1:3], reverse=True)
        list_poss_matches.append(razdel_poss_matches)
        razdel_poss_matches = []
        
    razdel_poss_matches.append((pos_ind, poss_matches))
razdel_poss_matches.sort(key=lambda x: x[1][0][1:3], reverse=True)
list_poss_matches.append(razdel_poss_matches)

In [333]:
thr_key = 0.5
for razd_poss_match in list_poss_matches:
    print(clrd(f"Раздел: {smeta[razd_poss_match[0][0]]['Section']}","yellow"))
    for poss_match in razd_poss_match:
        pos = smeta[poss_match[0]]
        print(clrd(f"#{poss_match[0]}", "magenta"))
        if(poss_match[1][0][1] > thr_key):
            print(clrd("Вероятно ключевая позиция","green","on_magenta")) 
            
        print(clrd(f"ИМЯ ПОЗИЦИИ ->>  {pos['Name']} \n {pos['Section']}","red"))
        if(poss_match[1][0][0]):
            print(clrd(f"match_coefficient: {poss_match[1][0][1:3]}","green"))
            print(df.iloc[poss_match[1][0][0]])

        else:
            print("Не найдено")
        print(clrd("-"*50,"magenta"))
    print(clrd(" <<< ГРАНИЦА РАЗДЕЛА >>> "*5,"cyan"))

Раздел: Раздел: Территория ЦАК
#3
Вероятно ключевая позиция
ИМЯ ПОЗИЦИИ ->>  Посадка кустарников с комом земли в живую изгородь - однорядная (без стоимости кустарников) 
 Раздел: Территория ЦАК
match_coefficient: (1.0, 0.49056603773584906)
СПГЗ                 Посадка кустарников на особо охраняемых природных территориях, штука
КПГЗ              02.03.05 СТРОИТЕЛЬСТВО НА ОСОБО ОХРАНЯЕМЫХ ПРИРОДНЫХ ТЕРРИТОРИЯХ (ООПТ)
Ключевые слова                                                          посадк*, кустарн*
Name: 865, dtype: object
--------------------------------------------------
#2
Вероятно ключевая позиция
ИМЯ ПОЗИЦИИ ->>  Посадка кустарников с комом земли в живую изгородь - однорядная (без стоимости кустарников) существующие 
 Раздел: Территория ЦАК
match_coefficient: (1.0, 0.4651162790697675)
СПГЗ                 Посадка кустарников на особо охраняемых природных территориях, штука
КПГЗ              02.03.05 СТРОИТЕЛЬСТВО НА ОСОБО ОХРАНЯЕМЫХ ПРИРОДНЫХ ТЕРРИТОРИЯХ (ООПТ)
Ключевые слов